## code 1

In [1]:
import time
import os
import sys
import pybullet as p
from cairo_simulator.core.utils import ASSETS_PATH
from cairo_simulator.core.log import Logger
from cairo_simulator.core.link import *
from cairo_simulator.core.simulator import Simulator, SimObject
from manipulatorsMOD import SawyerMOD
from cairo_simulator.devices.manipulators import Sawyer
from utils import load_configuration, save_config_to_configuration_file, manual_control, create_cuboid_obstacle
from pybullet_utils import bullet_client as bc
from pybullet_utils import urdfEditor as ed
import pybullet_data
import time

def main():
    use_ros = False
    use_real_time = True
    logger = Logger()
    sim = Simulator(logger=logger, use_ros=use_ros, use_real_time=use_real_time) # Initialize the Simulator
    p.setGravity(0,0,-9.81)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
    #added numSolverIterations modification to hopefully increase "stiffness" of held block
    p.setPhysicsEngineParameter(enableFileCaching=0)
    #p.setPhysicsEngineParameter(numSolverIterations=100, numSubSteps=10) #numSolverIterations=100, numSubSteps=10) #
    p.setPhysicsEngineParameter(solverResidualThreshold=1e-30)

    ground_plane = SimObject("Ground", "plane.urdf", [0,0,0])
    table = SimObject('table', 'NEWtable.urdf',  (0.9, 0.1, .47),(1.5708*2,0,0),fixed_base=1)
    #print(p.getNumJoints(table.get_simulator_id()))
    
    
    p.configureDebugVisualizer(p.COV_ENABLE_GUI,0) #disable explorer and camera views
    #Point the camera at the robot at the desired angle and distance
    #p.resetDebugVisualizerCamera( cameraDistance=1.5, cameraYaw=-30, cameraPitch=-30, cameraTargetPosition=[0.0, 0.0, 0.25])  
    p.resetDebugVisualizerCamera( cameraDistance=1.3, cameraYaw=92, cameraPitch=-37, 
                                 cameraTargetPosition=[-0.001, 0.03, 0.03])  

    
    
    #block = SimObject('block1', '1block.urdf', (0.69, -0.2, .55))
    
    #remember, for future objects, enable concave features in the URDF files using pybullet quckstart guide, like:
    #https://github.com/bulletphysics/bullet3/blob/master/data/samurai.urdf
    #sim_obj0 = SimObject('hole1', '1block.urdf',  (0.69, -0.2, .530),(0,0,0),fixed_base=0) 
    sim_obj1 = SimObject('hole1', '1.5hole.urdf',  (0.69, 0.1, .530),(0,0,0),fixed_base=1)  #1.5708 for 90 deg rotation
    sim_obj2 = SimObject('hole2', '1.25hole.urdf', (0.69, 0.3, .530),(0,0,0),fixed_base=1)
    sim_obj3 = SimObject('hole3', '1.15hole.urdf', (0.69, 0.5, .530),(0,0,0),fixed_base=1)    
    sim_obj4 = SimObject('hole1', 'C_hole.urdf',  (0.69, -0.5, .530),(0,0,0),fixed_base=1) 
    
    
    #sim_obj6 = SimObject('1block', '1block.urdf', (0.69, 0.1, .8),(0,0,0),fixed_base=0)
    
    #robotfilelocation=ASSETS_PATH + 'sawyer_description/urdf/sawyer_static.urdf'
    #sawyer_robot = Sawyer("sawyer0", [0, 0, 0.8], fixed_base=1)
    
    sawyer_robot = SawyerMOD(robot_name="sawyer0"
                           ,position=[0, 0, 0.8], fixed_base=1)
    
    robotID=sawyer_robot.get_simulator_id() #numeric code for robot
    

    dofindex=[3, 8, 9, 10, 11, 13, 16]
    #to display all joint information
                             
    """
    print(sawyer_robot._arm_dof_indices) #[3, 8, 9, 10, 11, 13, 16]  #these correspond to the links
    print(sawyer_robot._arm_dof_names) #['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
    print(get_joint_info_by_name(7,"right_j6"))
    for x in dofindex:
        print(get_joint_info(6,x))
        print("  ")
    for i in range(p.getNumJoints(robotID)):
        j_info = p.getJointInfo(robotID, i)
        print(j_info)
    """
    #enable force/torque sensors at all joints
    for x in dofindex:
        p.enableJointForceTorqueSensor(robotID,x,enableSensor=1)  #args: bodyID#, jointIndex,enablesensor
    
    orientation= [0,1,0,0] #quaternion hand pointing down..ish?
    xoffset=0
    yoffset=0
    
    #initial positioning of arm
    
    #jointPositions = [0.4945952894928332,0.36252717211583246,-1.0367314784884951,0.7450178202705928,-1.6496244258385506,-0.9285887664326956,-0.016547220371921873]
    jointPositions = [0.5009041352157391,0.2951560129283386,-1.2254625531233645,0.8216155636392285,-1.5183997750913025,-1.096785632443942,-0.08396404586438821]
    initialjointPositions=jointPositions
    #print("joints",range(p.getNumJoints(robotID)))     
    #gripperdof=list(sawyer_robot._gripper_dof_indices)
    #fingerPositions=list(sawyer_robot.get_gripper_pct_finger_positions(1))  
    
    for y in range(len(dofindex)):
        p.resetJointState(robotID, dofindex[y], jointPositions[y])
    """    
    dofindex.extend(gripperdof) 
    jointPositions.extend(fingerPositions)

    for y in range(len(dofindex)):
        p.resetJointState(robotID, dofindex[y], jointPositions[y])   
    """    
 
    
    joint_config2 = [0.69+xoffset, 0.1+yoffset, 0.9]
    joint_config = sawyer_robot.solve_inverse_kinematics(joint_config2,orientation)
    
    sawyer_robot.move_to_joint_pos(joint_config) 
    
    
    
    
    
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    
    key_w = ord('k') #y up
    key_a = ord('u') #x down
    key_s = ord('h') #y down
    key_d = ord('j') #x up
    
    
    
    key_ccw = ord('n') #gripper counterclockwise rotation (yaw)
    key_cw = ord('m') #gripper clockwise rotation(yaw)
    
    key_pitchup = ord('d') #gripper counterclockwise rotation (yaw)
    key_pitchdown = ord('c') #gripper clockwise rotation(yaw)
    key_rollleft = ord('z') #gripper counterclockwise rotation (yaw)
    key_rollright = ord('x') #gripper clockwise rotation(yaw)
    
    

    key_o = ord('o') #z up
    key_l = ord('l') #z downkey_8 = ord('8')
    key_5 = ord('5')
    
    key_7 = ord('7')
    key_8 = ord('8') #print all 6dof force/torque info
    key_9 = ord('9') #print info
    key_p = ord('p') 
    
    loop_count = 0 # Loop until someone shuts us down
    xcurrent=0.69
    ycurrent=0.1
    zcurrent=0.9
    # the center of 1.5 hole is roughly 0.71  0.08 OR   ~0.78 in x and y
    a=0
    b=1
    c=0
    d=0
    orientation= [a,b,c,d] #quaternion hand pointing down..ish?
    
    
    print("HK to jog_X         UJ to jog_Y           OL to jog_Z")
    print("ZX to roll          DC to Pitch           NM to Yaw(rotate end-effector)")
    print("8 for all joint force/torque, 9 for J16 and XYZ location")
    while True:
            xoffset=0
            yoffset=0
            
            keys = p.getKeyboardEvents()
            if key_w in keys and keys[key_w] & p.KEY_WAS_TRIGGERED:
                ycurrent+=0.01
                xyz = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_a in keys and keys[key_a] & p.KEY_WAS_TRIGGERED:
                xcurrent-=0.01
                xyz = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_s in keys and keys[key_s] & p.KEY_WAS_TRIGGERED:
                ycurrent-=0.01
                xyz = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_d in keys and keys[key_d] & p.KEY_WAS_TRIGGERED:
                xcurrent+=0.01
                xyz = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
                
                
            if key_o in keys and keys[key_o] & p.KEY_WAS_TRIGGERED:
                zcurrent+=0.01
                xyz = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_l in keys and keys[key_l] & p.KEY_WAS_TRIGGERED:
                zcurrent-=0.01
                xyz = [xcurrent, ycurrent, zcurrent]
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)  
                
            if key_cw in keys and keys[key_cw] & p.KEY_WAS_TRIGGERED:
                xyz = [xcurrent, ycurrent, zcurrent]
                a+=0.03 #a= yaw- use this! c= roll d= pitch
                orientation= [a,b,c,d] #[0,1,0,0] default #quaternion hand pointing down..ish?
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)

            if key_ccw in keys and keys[key_ccw] & p.KEY_WAS_TRIGGERED:
                xyz = [xcurrent, ycurrent, zcurrent]
                a-=0.03 #a= yaw- use this! c= roll d= pitch
                orientation= [a,b,c,d] #[0,1,0,0] default #quaternion hand pointing down..ish?
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
                
            if key_pitchup in keys and keys[key_pitchup] & p.KEY_WAS_TRIGGERED:
                xyz = [xcurrent, ycurrent, zcurrent]
                d-=0.03 #a= yaw- use this! c= roll d= pitch
                orientation= [a,b,c,d] #[0,1,0,0] default #quaternion hand pointing down..ish?
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)

            if key_pitchdown in keys and keys[key_pitchdown] & p.KEY_WAS_TRIGGERED:
                xyz = [xcurrent, ycurrent, zcurrent]
                d+=0.03 #a= yaw- use this! c= roll d= pitch
                orientation= [a,b,c,d] #[0,1,0,0] default #quaternion hand pointing down..ish?
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)
            if key_rollleft in keys and keys[key_rollleft] & p.KEY_WAS_TRIGGERED:
                xyz = [xcurrent, ycurrent, zcurrent]
                c-=0.03 #a= yaw- use this! c= roll d= pitch
                orientation= [a,b,c,d] #[0,1,0,0] default #quaternion hand pointing down..ish?
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)

            if key_rollright in keys and keys[key_rollright] & p.KEY_WAS_TRIGGERED:
                xyz = [xcurrent, ycurrent, zcurrent]
                c+=0.03 #a= yaw- use this! c= roll d= pitch
                orientation= [a,b,c,d] #[0,1,0,0] default #quaternion hand pointing down..ish?
                joint_config = sawyer_robot.solve_inverse_kinematics(xyz,orientation)
                sawyer_robot.move_to_joint_pos(joint_config)    
                
                
                
            if key_5 in keys and keys[key_5] & p.KEY_WAS_TRIGGERED:
                #Get camera position 
                #last sets of values are: yaw, pitch, dist,target(list of 3 floats)
                #plug values into resetDebugVisualizerCamera command at the top to set the initial camera view. 
                print(p.getDebugVisualizerCamera())
                
          
                """
                dofindex=[3, 8, 9, 10, 11, 13, 16]
                print("Moving down in Z until impact")
                for x in dofindex:
                    jointinfo=p.getJointState(robotID,int(x)) 
                    #rounded6dof = [round(num, 3) for num in jointinfo[2]]
                    print("J", x , " ", jointinfo[0])   
                """
                    
            if key_7 in keys and keys[key_7] & p.KEY_WAS_TRIGGERED:
                
                dofindex=[3, 8, 9, 10, 11, 13, 16]
                print("All Joint positions")
                for x in dofindex:
                    jointinfo=p.getJointState(robotID,int(x)) 
                    #rounded6dof = [round(num, 3) for num in jointinfo[2]]
                    print("J", x , " ", jointinfo[0]) 
        
            if key_8 in keys and keys[key_8] & p.KEY_WAS_TRIGGERED:
                
                dofindex=[3, 8, 9, 10, 11, 13, 16]
                print("All Joint Reaction Forces")
                for x in dofindex:
                    jointinfo=p.getJointState(robotID,int(x))  
                    #jointPosition,jointVelocity,(6dof)jointReactionForces, appliedJointMotorTorque
                    rounded6dof = [round(num, 3) for num in jointinfo[2]]
                    print("J", x , " ", rounded6dof) 
                
            if key_9 in keys and keys[key_9] & p.KEY_WAS_TRIGGERED:
                print(" ")
                print("location", xcurrent,"  ",ycurrent,"  ",zcurrent)
                #print("enter joint number [3, 8, 9, 10, 11, 13, 16]")
                #a = input()
                #jointinfo=p.getJointState(7,16) 
                #jointinfo=p.getJointState(7,int(a))
                jointinfo=p.getJointState(robotID,16) 
                rounded6dof = [round(num, 3) for num in jointinfo[2]]
                print("J 16",rounded6dof)
                #print(jointinfo)
                
            #test function    
            if key_p in keys and keys[key_p] & p.KEY_WAS_TRIGGERED:
                print(" ")
                print("test")
                dofindex=[3, 8, 9, 10, 11, 13, 16]
                state=sawyer_robot.get_current_joint_states() 
                state2=[]
                for i in dofindex:
                    state2.append(p.getJointState(robotID,i)[0])
                print("state")
                print(state)
                print("state2")
                print(state2)                  
                                  
                sawyer_robot.move_to_joint_pos(state2) 
                
            
                
            time.sleep(0.2)
          
    #manual_control(sawyer_robot)    
    try:
        while True:
            sim.step()
    except KeyboardInterrupt:
        p.disconnect()
        sys.exit(0)

In [ ]:
main()


HK to jog_X         UJ to jog_Y           OL to jog_Z
ZX to roll          DC to Pitch           NM to Yaw(rotate end-effector)
8 for all joint force/torque, 9 for J16 and XYZ location


In [3]:
jointstate=[0.49493610055318166, 0.36099143680909324, -1.0340115168334552,
            0.7453425112185333, -1.6540825650620934, -0.9272504160528141,
            -0.015913173027092584, 0.022526081455362414, -0.020833398856934377]
print(jointstate[6])



-0.015913173027092584


In [22]:
jointPositions= [0.4945952894928332, 0.36252717211583246, -1.0367314784884951, 
                 0.7450178202705928, -1.6496244258385506, -0.9285887664326956, -0.016547220371921873]
fingerPositions=(0.0104165, -0.0104165)
listfingerPositions=list(fingerPositions)

#jointandGripperPositions= jointandGripperPositions.extend(fingerPositions[1])
print("initial",jointPositions)
#print(listfingerPositions[0])
print(fingerPositions)
print("")
jointPositions.extend(listfingerPositions)
print("final",jointPositions)
print("a")


initial [0.4945952894928332, 0.36252717211583246, -1.0367314784884951, 0.7450178202705928, -1.6496244258385506, -0.9285887664326956, -0.016547220371921873]
(0.0104165, -0.0104165)

final [0.4945952894928332, 0.36252717211583246, -1.0367314784884951, 0.7450178202705928, -1.6496244258385506, -0.9285887664326956, -0.016547220371921873, 0.0104165, -0.0104165]
a


In [ ]:

import pybullet as p
import time
import math

import pybullet_data
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.loadURDF("plane.urdf")
cubeId = p.loadURDF("cube_small.urdf", 0, 0, 1)
p.setGravity(0, 0, -10)
p.setRealTimeSimulation(1)
cid = p.createConstraint(cubeId, -1, -1, -1, p.JOINT_FIXED, [0, 0, 0], [0, 0, 0], [0, 0, 1])
print(cid)
print(p.getConstraintUniqueId(0))
a = -math.pi
while 1:
  a = a + 0.01
  if (a > math.pi):
    a = -math.pi
  time.sleep(.01)
  p.setGravity(0, 0, -10)
  pivot = [a, 0, 1]
  orn = p.getQuaternionFromEuler([a, 0, 0])
  p.changeConstraint(cid, pivot, jointChildFrameOrientation=orn, maxForce=50)

p.removeConstraint(cid)
        

1
1


In [ ]:
    #___________________#Test Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    kuka = p0.loadURDF('1.5hole.urdf')
    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    
    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0
    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]
    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    #link index = joint index
    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0)  
    
    #_______________________ end URDF combining test

## code 2


In [ ]:
    #___________________#Combining URDF's
    #https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/combineUrdf.py
    p0 = bc.BulletClient(connection_mode=p.DIRECT)
    p0.setAdditionalSearchPath(pybullet_data.getDataPath())
    p1 = bc.BulletClient(connection_mode=p.DIRECT)
    p1.setAdditionalSearchPath(pybullet_data.getDataPath())

    #husky = p1.loadURDF('1block.urdf', flags=p0.URDF_USE_IMPLICIT_CYLINDER)  #this flag appears to have no impact
    husky = p1.loadURDF('1block.urdf')
    kuka = p0.loadURDF('1.5hole.urdf')

    ed0 = ed.UrdfEditor()
    ed0.initializeFromBulletBody(husky, p1._client)
    ed1 = ed.UrdfEditor()
    ed1.initializeFromBulletBody(kuka, p0._client)

    parentLinkIndex = 0

    jointPivotXYZInParent = [0, 0, 0]
    jointPivotRPYInParent = [0, 0, 0]

    jointPivotXYZInChild = [0, 0, 0]
    jointPivotRPYInChild = [0, 0, 0]

    newjoint = ed0.joinUrdf(ed1, parentLinkIndex, jointPivotXYZInParent, jointPivotRPYInParent,
                            jointPivotXYZInChild, jointPivotRPYInChild, p0._client, p1._client)
    newjoint.joint_type = p0.JOINT_FIXED

    ed0.saveUrdf("combined.urdf")
    #pgui = bc.BulletClient(connection_mode=p.GUI)
    #pgui.configureDebugVisualizer(pgui.COV_ENABLE_RENDERING, 0)
    
    orn = [0, 0, 0, 1]
    #ed0.createMultiBody([0, 0, 0], orn, pgui._client)  #doesn't work
    
    
    #ed0.createMultiBody([(0.69, -0.1, .530)], orn)  #works
    sim_obj3 = SimObject('hole3',"combined.urdf", (0.69, -0.4, .530),(0,0,0),fixed_base=0) 


0
1
p0.getNumBodies()= 1
p1.getNumBodies()= 1
fileNames= ['./1.5in squarehole_C.SLDPRT/meshes/1.5in squarehole_C.SLDPRT.STL']


 aaa


ehllo aaa
